In [92]:
! pip install tensorflow

     -------------------------------------- 266.3/266.3 MB 3.6 MB/s eta 0:00:00
     ---------------------------------------- 3.7/3.7 MB 10.4 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 11.9 MB/s eta 0:00:00
     ---------------------------------------- 57.5/57.5 kB 3.1 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 10.5 MB/s eta 0:00:00
     -------------------------------------- 439.2/439.2 kB 6.8 MB/s eta 0:00:00
     --------------------------------------- 23.2/23.2 MB 10.9 MB/s eta 0:00:00
     ---------------------------------------- 65.5/65.5 kB 3.7 MB/s eta 0:00:00
     ---------------------------------------- 6.0/6.0 MB 11.6 MB/s eta 0:00:00
     ------------------------------------- 895.9/895.9 kB 11.4 MB/s eta 0:00:00
     -------------------------------------- 126.5/126.5 kB 7.3 MB/s eta 0:00:00
     ---------------------------------------- 177.2/177.2 kB ? eta 0:00:00
     ------------------------------------- 781.3/

In [1]:
import pandas as pd
# import MySQLdb
import sklearn

# import konlpy
# from konlpy.tag import Twitter

import gensim
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence
from gensim import corpora, models
from gensim.models import LdaMulticore
from gensim.models import Word2Vec
from gensim.corpora import Dictionary, MmCorpus
from gensim.models import KeyedVectors

# import pyLDAvis
# import pyLDAvis.gensim
import pickle

# from wordcloud import WordCloud
# from collections import Counter

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE


from os import path
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, value
# output_notebook()

In [5]:
hiphop = pd.read_csv('melon_trot_1_3500.csv')
hiphop.head()

,제목,가사,가수,발매일,좋아요수,장르
0,이제 나만 믿어요,무얼 믿은 걸까 부족했던 내게서\n나조차 못 믿던 내게 여태 머문 사람\n무얼 봤던...,임영웅,2020.04.03,"164,747",성인가요/트로트
1,보금자리,그대 사랑이 나였음 좋겠다\n아무것도 필요 없어요\n든든한 품에 안겨 잠들고 싶어라...,임영웅,2022.05.02,"43,816",성인가요/트로트
2,찐이야,찐찐찐찐 찐이야 완전 찐이야\n진짜가 나타났다 지금\n찐찐찐찐 찐이야 완전 찐이야\...,영탁,2020.03.13,"66,514",성인가요/트로트
3,니가 왜 거기서 나와 (Narr. 고은아),어디야\n집이야 피곤해서 일찍 자려구\n아 그래 잠깐 볼랬더니\n오늘 피곤했나 보네...,영탁,2018.10.21,"61,103",성인가요/트로트
4,별빛 같은 나의 사랑아,당신이 얼마나 내게\n소중한 사람인지\n세월이 흐르고 보니\n이제 알 것 같아요\n...,임영웅,2021.03.09,"76,133",성인가요/트로트


In [6]:
hiphop.columns

Index(['제목', '가사', '가수', '발매일', '좋아요수', '장르'], dtype='object')

In [7]:
USE_PREMADE_TEXT = False

text_filepath = 'word2vec/all_lyrics_text.txt'
if not USE_PREMADE_TEXT:
    with open(text_filepath, 'w', encoding='utf-8') as f:
        for lyrics in hiphop['가사'].values:
            if pd.isnull(lyrics): # null값 있다면 그 다음으로 넘어감
                continue
            f.write(lyrics + '\n')
else:
    assert path.exists(text_filepath)

In [8]:
USE_PREMADE_BIGRAM_MODEL = False

all_bigram_model_filepath = 'word2vec/all_bigram_model'
all_sentences_normalized_filepath = 'word2vec/all_lyrics_text.txt'

all_unigram_sentences = LineSentence(all_sentences_normalized_filepath)

if not USE_PREMADE_BIGRAM_MODEL:    
    
    all_bigram_model = Phrases(all_unigram_sentences) #phrase냐 아니냐를 판단해줌
    all_bigram_model.save(all_bigram_model_filepath)
    
else:
    all_bigram_model = Phrases.load(all_bigram_model_filepath)

In [9]:
USE_PREMADE_BIGRAM_SENTENCES = False

all_bigram_sentences_filepath = 'word2vec/all_sentences_for_word2vec.txt'

if not USE_PREMADE_BIGRAM_SENTENCES:
    
    with open(all_bigram_sentences_filepath, 'w', encoding='utf-8') as f:
        for unigram_sentence in all_unigram_sentences:
            all_bigram_sentence = all_bigram_model[unigram_sentence]
            f.write(' '.join(all_bigram_sentence) + '\n')
else:
    assert path.exists(all_bigram_sentences_filepath)

In [10]:
USE_PREMADE_WORD2VEC = False

all2vec_filepath = 'word2vec/all_word2vec_model'

if not USE_PREMADE_WORD2VEC:
    
    lyrics_for_word2vec = LineSentence(all_bigram_sentences_filepath)

    all2vec = Word2Vec(lyrics_for_word2vec, vector_size=100, window=5, min_count=1, sg=1)
    # sg=0 cbow 1=Skip-Gram Model
    # 100차원으로 가져옴 / 보통 20~100 정도
    # window = 5 앞 5개, 뒤 5개 단어를 보겠다는 뜻
    # window size 작을수록 문법적인 의미가 너무 중요해짐, 클수록 주제 지향적으로 문맥적인 정보를 많이 담게 됨    
    for _ in range(9):
        all2vec.train(lyrics_for_word2vec, total_examples=81085, epochs=1)
        # 9139곡 가사에 총 789847 문장이 있다는 것 명시
        
    all2vec.save(all2vec_filepath)
else:
    all2vec = Word2Vec.load(all2vec_filepath)
all2vec.init_sims()

C:\Users\khyj0\AppData\Local\Temp\ipykernel_18228\1412957348.py:21: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  all2vec.init_sims()


In [11]:
all2vec_filepath = 'word2vec/all_word2vec_model'
all2vec = Word2Vec.load(all2vec_filepath)

In [12]:
all2vec.wv.most_similar(positive=['사랑'], topn=5)

[('얘기_들려줬지', 0.8526408672332764),
 ('어릿광대의_서글픈', 0.8409197926521301),
 ('허락한', 0.8371437191963196),
 ('지워지지않는', 0.8337648510932922),
 ('사랑하시렵니까', 0.8337077498435974)]

In [13]:
all2vec.wv.most_similar(positive=['엄마'], topn=5)

[('눈매에', 0.8991658687591553),
 ('나는야_당신의', 0.8922415971755981),
 ('눈물인가', 0.8897567987442017),
 ('친정엄마', 0.8883345723152161),
 ('컷이라도', 0.8877487182617188)]

In [14]:
a = pd.DataFrame(all2vec.wv.index_to_key)
a.head()

,0
0,내
1,그
2,사랑
3,내가
4,나는


In [15]:
len (a)

32851

In [16]:
words = []
cnt = all2vec.wv.index_to_key[0]
all2vec.wv.get_vecattr(cnt, "count")


3650

In [17]:
a, b = enumerate(all2vec.wv.key_to_index)

ValueError: too many values to unpack (expected 2)

In [18]:
%%time
words = []
for i in (range(32851)):
    cnt_num = all2vec.wv.index_to_key[i]
    cnt = all2vec.wv.get_vecattr(cnt_num, "count")
    if cnt > 100:
        words.append(a[0][i]) # 횟수 100회 초과하는 단어만 리스트화
    i += 1

CPU times: total: 46.9 ms
Wall time: 96.9 ms


In [19]:
len (words)

193

In [20]:
X = []
for i in words:
    X.append(all2vec.wv[i])

In [21]:
type (X)

list

In [22]:
X2 = pd.DataFrame(X, index = words)

In [23]:
X2.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
내,-0.136166,0.366624,1.034507,1.433129,-0.514249,-0.298968,0.879357,1.351689,-1.294456,-0.493064,...,-0.481747,-0.471407,-0.028197,0.012252,1.081335,0.171065,-1.172329,-1.360425,-0.586285,0.998504
그,0.208582,0.517061,0.819431,0.818019,0.654620,-0.805044,0.646501,1.413978,-1.302137,-0.306315,...,0.320788,-0.193923,0.136352,-0.298862,0.322760,0.041108,0.493559,-0.702168,-0.784199,0.078243
사랑,-0.367977,0.569756,-0.145142,-0.781577,0.019094,0.153369,-0.215093,1.892271,-0.320322,0.681078,...,0.063266,0.140259,0.286728,-0.030319,0.443175,0.322728,0.007324,-0.874286,0.623615,0.070719
내가,-1.360877,1.106440,-0.188100,0.168360,0.812520,-0.721575,0.284537,1.743519,-0.041589,-0.961685,...,-0.022407,-0.083306,0.013059,-0.332200,1.154214,-0.729024,0.516737,-0.481769,-0.576209,-0.134269
나는,-0.335103,-0.325015,-0.978000,0.705873,0.295354,-0.351638,0.327134,2.089930,-0.537545,0.199769,...,0.736009,-0.377910,1.495826,0.605818,1.066280,-0.900749,0.506734,0.673915,0.262671,0.564055


In [25]:
USE_PREMADE_TSNE = False

tsne_filepath = 'word2vec/tsne.pkl'

if not USE_PREMADE_TSNE:
    
    tsne = TSNE(random_state=0)
    tsne_points = tsne.fit_transform(X2)
    with open(tsne_filepath, 'wb') as f:
        pickle.dump(tsne_points, f)
else:
    with open(tsne_filepath, 'rb') as f:
        tsne_points = pickle.load(f)

tsne_df = pd.DataFrame(tsne_points, index=X2.index, columns=['x_coord', 'y_coord'])
tsne_df['word'] = tsne_df.index

C:\Users\khyj0\anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
C:\Users\khyj0\anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


In [34]:
# prepare the data in a form suitable for bokeh.
plot_data = ColumnDataSource(tsne_df)

# create the plot and configure it
tsne_plot = figure(title="t-SNE Word Embeddings",
                   plot_width = 800,
                   plot_height = 800,
                   active_scroll='wheel_zoom'
                  )

# add a hover tool to display words on roll-over
tsne_plot.add_tools( HoverTool(tooltips = '@word') )

tsne_plot.circle('x_coord', 'y_coord', source=plot_data,
                 color='red', line_alpha=0.2, fill_alpha=0.1,
                 size=10, hover_line_color='orange')

# adjust visual elements of the plot
tsne_plot.xaxis.visible = False
tsne_plot.yaxis.visible = False
tsne_plot.grid.grid_line_color = None
tsne_plot.outline_line_color = None

# show time!
show(tsne_plot);

In [35]:
import collections
import numpy as np
import gensim
from gensim.models import Phrases
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

class TextLoader:
    def __init__(self, path):
        with open(path, "r", encoding='utf-8') as _file:
            self.text = _file.read().split()
            # self.text = self.text[:10000]

        self.song2vec = Word2Vec.load("word2vec/all_word2vec_model")
        self.vocab, self.words = self.build_vocab()

        self.X = self.text[:] # 텍스트 파일 전체를 복사 - self.text 와 같은 의미
        self.y = [self.text[0]] + self.text[1:] # 

    def build_vocab(self):
        vocab_inv = list(self.song2vec.wv.index_to_key) # key 값을 리스트화 / 글자만
        vocab = {x: i for i, x in enumerate(vocab_inv)} # 0,1,2 등 인덱스와 단어를 dict 로 매칭시켜놓음
        return vocab, vocab_inv

    def next_batch(self, batch_size, seq_length):
        start = np.random.randint(0, len(self.X)-batch_size*seq_length) # 랜덤으로 위치를 정함 - 끝의 값을 구하면 안됨 / 시작 위치를 글자를 다 배치사이즈와 시퀀스렝스로 구함 // 마지막까지는 안가겠다는 뜻
        end   = start + batch_size*seq_length # 몇 단어를 가져올지

        X_words = self.X[start:end]# 말그대로 글자
        y_words = self.y[start:end]

        X_idx = np.empty((batch_size, seq_length), dtype=np.int64) # 글자의 인덱스
        y_idx = np.empty((batch_size, seq_length), dtype=np.int64)
        X_wv = np.empty((batch_size, seq_length, 100)) # 글자의 word2vec 
        y_wv = np.empty((batch_size, seq_length, 100))
        # 위에서 만들어준 자리에 따라 (저장공간 설정하는 과정) 아래에서 for 문을 돌며 값을 가져옴 / 그냥 하면 안되는 이유 : append는 느림, numpy의 경우에는 1 2 3 4 붙어있어야 함 / 5를 넣는다 하면 이걸 어딘가 복사해서 5를 붙여야 함.
        for i in range(batch_size):
            for j in range(seq_length):
                X_idx[i, j] = self.vocab[X_words[i*seq_length+j]]
                y_idx[i, j] = self.vocab[y_words[i*seq_length+j]]

                X_wv[i, j] = self.song2vec.wv[X_words[i*seq_length+j]]
                y_wv[i, j] = self.song2vec.wv[y_words[i*seq_length+j]]

        return X_wv, X_idx, y_wv, 

In [36]:
import os
import numpy as np
import torch
import torch.nn as nn
import tensorflow as tf
# import tensorflow.contrib.rnn as rnn
# import tensorflow.contrib.layers as layers

In [37]:
num_layers  = 3
hidden_size = 512
batch_size  = 1 # 1글자
# max_length  = 1 # 1로 하는것으로~

loader = TextLoader("word2vec/all_sentences_for_word2vec.txt")
vocab_size = len(loader.vocab)

In [44]:

import torch
import torch.nn as nn

X = torch.zeros((batch_size, vocab_size)) # assuming you have already loaded your data and created the tensors

class Model(nn.Module):
    def __init__(self, hidden_size, num_layers, vocab_size):
        super(Model, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.vocab_size = vocab_size
        self.lstm = nn.LSTM(input_size=vocab_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = out.reshape(-1, self.hidden_size)
        out = self.fc(out)
        return out

model = Model(hidden_size, num_layers, vocab_size)
outputs = model(X)

RuntimeError: For unbatched 2-D input, hx and cx should also be 2-D but got (3-D, 3-D) tensors

In [39]:
# 시작 글자 생성
sentence = ["안녕", "나는", "너야"]
print(sentence)
print("Start with:", " ".join(sentence))

saver = tf.train.Saver()
sess_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
with tf.Session(config=sess_config) as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, "char-rnn_230000")
    
    # [배치사이즈, max_length, 100]
    vec = np.empty((1, len(sentence), 100)) # 시작 글자를 주고 다음 단어를 예측 - 시작으로 준 것을 전부 다 입력으로 넣겠다
    for i, word in enumerate(sentence):
        vec[:, i, :] = loader.song2vec.wv[word]
    
    # 매 이터레이션마다 글자 하나씩 생성
    state = sess.run(states, feed_dict={X: vec}) #입력단어 sentence 이후에 들어올 단어를 예측
    for i in range(15): # for문을 돌면서 풀어헤치는 중!!
        vec = loader.song2vec.wv[sentence[-1]].reshape(1, 1, 100)
        
        pred_char, state = sess.run([pred, states], 
            feed_dict={X: vec, initial_state: state}) # 원래 initial stete 는 0 이었으나
        # 이전 스텝에 갖고 있는 state 값을 다음 스텝에 넣어줌 // 입력단어들을 그 다음 스텝에 넣어주는 것임
        
        pred_char = loader.words[pred_char[0][-1]]
        sentence.append(pred_char)

for i, word in enumerate(sentence):
    print(word, end=" ")
    if (i+1) % 5 == 0:
        print()

['안녕', '나는', '너야']
Start with: 안녕 나는 너야


AttributeError: module 'tensorflow._api.v2.train' has no attribute 'Saver'